# Medical question and answer

In [ ]:
!pip install peft
!pip install datasets
!pip install --upgrade transformers
!pip install evaluate
!pip install rouge_score
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparing the Dataset.

In [ ]:
from datasets import load_dataset, DatasetDict

# Language Generation
data_wiki_general = load_dataset("medalpaca/medical_meadow_wikidoc")
data_patient_wiki = load_dataset("medalpaca/medical_meadow_wikidoc_patient_information")

def split_dataset(data):
  # Perform a train-test split
  data = data["train"].train_test_split(test_size=0.1)

  # Further split the training data into training and validation sets
  test_data = data["test"]
  train_data = data["train"].train_test_split(test_size=0.1)
  valid_data = train_data["test"]
  train_data = train_data["train"]
  # Print dataset splits
  print("Train data:", len(train_data))
  print("Validation data:", len(valid_data))
  print("Test data:", len(test_data))

  return train_data, valid_data, test_data

train_data_wiki, valid_data_wiki, test_data_wiki = split_dataset(data_wiki)
# Create a combined DatasetDict
data_wiki = DatasetDict({
    "train": train_data_wiki,
    "validation": valid_data_wiki,
    "test": test_data_wiki
})
train_data_patient, valid_data_patient, test_data_patient = split_dataset(data_patient_wiki)
data_patient_wiki = DatasetDict({
    "train": train_data_patient,
    "validation": valid_data_patient,
    "test": test_data_patient
})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

medical_meadow_wikidoc.json:   0%|          | 0.00/10.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

medical_meadow_wikidoc_patient_info.json:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5942 [00:00<?, ? examples/s]

Train data: 8100
Validation data: 900
Test data: 1000
Train data: 4812
Validation data: 535
Test data: 595


In [ ]:
print(data_patient_wiki)

print(data_patient_wiki['train'][10])

print(data_patient_wiki['train'][10]['input'])
print(data_patient_wiki['train'][10]['output'])

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 4812
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 535
    })
    test: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 595
    })
})
{'input': 'Who is at highest risk for Syndrome of inappropriate antidiuretic hormone ?', 'output': 'Central nervous system disease: Brain tumors, trauma, infection such as meningitis and encephalitis, Guillain-Barré syndrome, delirium tremens, multiple sclerosis Pulmonary disease: Small cell lung cancer, pneumonia, chronic obstructive pulmonary disease, lung abscess, tuberculosis, cystic fibrosis, positive-pressure ventilation Cancer: Lung cancer, pancreas cancer, thymoma, ovarian cancer, lymphoma, laryngeal cancer, Nasopharyngeal carcinoma, bladder cancer, prostate cancer, breast cancer, melanoma. Drugs: Nonsteroidal anti-inflammatory drugs, nicotine, diure

In [ ]:
from datasets import Dataset

def tokenization_dataset(dataset, tokenizer, max_length=512):
    # Tokenize the dataset
    def tokenize_function(examples):
        # For Question Answering, we concatenate question and answer and also set start/end positions
        questions = examples["input"]
        answers = examples["output"]

        encodings = tokenizer(questions, answers, padding="max_length", truncation=True, max_length=max_length)

        start_positions = []
        end_positions = []

        # Calculate the start and end positions
        for idx, example in enumerate(examples["input"]):
            answer_text = answers[idx]  # Get the answer text for this example
            answer_start = example.find(answer_text)  # Search for the answer in the input text

            # Check if answer is found and not empty
            if answer_start != -1 and answer_text:
                answer_end = answer_start + len(answer_text)

                start_token = encodings.char_to_token(idx, answer_start)
                end_token = encodings.char_to_token(idx, answer_end - 1)

                # Handle cases where char_to_token returns None
                if start_token is None:
                    start_token = 0  # or another suitable default value
                if end_token is None:
                    end_token = 0  # or another suitable default value

                start_positions.append(start_token)
                end_positions.append(end_token)
            else:
                # Handle cases where answer is not found or empty
                start_positions.append(0)  # or another suitable default value
                end_positions.append(0)  # or another suitable default value

        encodings["start_positions"] = start_positions
        encodings["end_positions"] = end_positions

        return encodings

    # Apply the tokenization to train, validation, and test sets
    train_data = dataset["train"].map(tokenize_function, batched=True)
    valid_data = dataset["validation"].map(tokenize_function, batched=True)
    test_data = dataset["test"].map(tokenize_function, batched=True)

    return train_data, valid_data, test_data

# Modeling and representation

In [ ]:
from transformers import (AutoModelForQuestionAnswering,
                          TrainingArguments,
                          Trainer,
                          AutoTokenizer,
                          GPT2ForQuestionAnswering,
                          T5ForQuestionAnswering)

distilBerttokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
distilBertQA = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilroberta-base")

GPT2tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
GPT2tokenizer.pad_token = GPT2tokenizer.eos_token
GPT2QA = GPT2ForQuestionAnswering.from_pretrained("openai-community/gpt2")

T5Tokenizer = AutoTokenizer.from_pretrained("t5-base")
T5QA = T5ForQuestionAnswering.from_pretrained("t5-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at t5-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data_distilBert, valid_data_distilBert, test_data_distilBert = tokenization_dataset(data_wiki,distilBerttokenizer)

train_data_GPT2, valid_data_GPT2, test_data_GPT2 = tokenization_dataset(data_wiki, GPT2tokenizer)

train_data_T5, valid_data_T5, test_data_T5 = tokenization_dataset(data_wiki, T5Tokenizer)

Map:   0%|          | 0/8100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(valid_data_distilBert['start_positions'])

print(valid_data_distilBert['end_positions'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Fine-Tuning.


TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING = {
    "t5": ["q", "v"],
    "mt5": ["q", "v"],
    "bart": ["q_proj", "v_proj"],
    "gpt2": ["c_attn"],
    "bloom": ["query_key_value"],
    "blip-2": ["q", "v", "q_proj", "v_proj"],
    "opt": ["q_proj", "v_proj"],
    "gptj": ["q_proj", "v_proj"],
    "gpt_neox": ["query_key_value"],
    "gpt_neo": ["q_proj", "v_proj"],
    "bert": ["query", "value"],
    "roberta": ["query", "value"],
    "xlm-roberta": ["query", "value"],
    "electra": ["query", "value"],
    "deberta-v2": ["query_proj", "value_proj"],
    "deberta": ["in_proj"],
    "layoutlm": ["query", "value"],
    "llama": ["q_proj", "v_proj"],
    "chatglm": ["query_key_value"],
    "gpt_bigcode": ["c_attn"],
    "mpt": ["Wqkv"],
}

In [ ]:
import peft
from peft import LoraConfig, get_peft_model, PeftModel

def define_model(model, lora_config, label_names):
    peft_model = get_peft_model(model, lora_config)
    # You can now pass label_names if needed
    peft_model.config.label_names = label_names
    print(peft_model.print_trainable_parameters())
    return peft_model

# When defining models
label_names = ["start_positions", "end_positions"]

# TARGET_MODULES
# https://github.com/huggingface/peft/blob/39ef2546d5d9b8f5f8a7016ec10657887a867041/src/peft/utils/other.py#L220
lora_config_distilRoberta = LoraConfig(
        r=4, #As bigger the R bigger the parameters to train.
        lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
        target_modules=["query", "value"], #You can obtain a list of target modules in the URL above.
        lora_dropout=0.05, #Helps to avoid Overfitting.
        bias="lora_only", # this specifies if the bias parameter should be trained.
        task_type="QUESTION_ANS"
    )

lora_config_GPT2 = LoraConfig(
        r=4, #As bigger the R bigger the parameters to train.
        lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
        target_modules=["c_attn"], #You can obtain a list of target modules in the URL above.
        lora_dropout=0.05, #Helps to avoid Overfitting.
        bias="lora_only", # this specifies if the bias parameter should be trained.
        task_type="QUESTION_ANS"
    )

lora_config_T5 = LoraConfig(
        r=4, #As bigger the R bigger the parameters to train.
        lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
        target_modules=["q", "v"], #You can obtain a list of target modules in the URL above.
        lora_dropout=0.05, #Helps to avoid Overfitting.
        bias="lora_only", # this specifies if the bias parameter should be trained.
        task_type="QUESTION_ANS"
    )

peft_distilRoberta = define_model(distilBertQA, lora_config_distilRoberta, label_names)
peft_GPT2 = define_model(GPT2QA, lora_config_GPT2, label_names)
peft_T5 = define_model(T5QA, lora_config_T5, label_names)

trainable params: 84,482 || all params: 81,604,612 || trainable%: 0.1035
None
trainable params: 176,642 || all params: 124,590,340 || trainable%: 0.1418
None
trainable params: 443,906 || all params: 223,348,996 || trainable%: 0.1987
None


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


# Model Evaluation

In [ ]:
#Creating the TrainingArgs
import transformers
from transformers import TrainingArguments, Trainer

import evaluate
import numpy as np

# Load all relevant metrics
squad_metric = evaluate.load("squad")
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")

def compute_metrics(p, tokenizer):
    predictions, labels = p
    print(f"Predictions: {predictions}")
    print(f"Labels: {labels}")

    # Convert predictions (logits) to ids
    pred_ids = np.argmax(predictions, axis=-1)  # Use argmax to get the predicted class/word id

    # Decode token predictions into text using the tokenizer
    pred_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in pred_ids]

    # Decode labels into reference texts (assuming labels are token ids, not structured data)
    ref_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels]

    # Print one example of Q&A
    example_idx = 0  # You can choose a specific index to print
    print(f"Example {example_idx + 1}:")
    print(f"Predicted Answer: {pred_texts[example_idx]}")
    print(f"Reference Answer: {ref_texts[example_idx]}")
    print()

    # Compute ROUGE and BLEU scores
    rouge_results = rouge_metric.compute(predictions=pred_texts, references=ref_texts)
    bleu_results = bleu_metric.compute(predictions=pred_texts, references=[[ref] for ref in ref_texts])

    # Compute SQuAD scores
    squad_preds = [{"id": str(i), "prediction_text": pred} for i, pred in enumerate(pred_texts)]
    squad_refs = [{"id": str(i), "answers": {"text": [ref]}} for i, ref in enumerate(ref_texts)]

    squad_results = squad_metric.compute(predictions=squad_preds, references=squad_refs)

    # Combine all results into one dictionary
    results = {
        "exact_match": squad_results["exact_match"],
        "f1": squad_results["f1"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "rougeLsum": rouge_results["rougeLsum"],
        "bleu": bleu_results["bleu"]
    }

    return results



from transformers import DataCollatorWithPadding
from torch.utils.data import default_collate
import torch

class QADataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.collator = DataCollatorWithPadding(tokenizer=tokenizer)

    def __call__(self, features):
        # Extract start and end positions separately
        start_positions = [f.pop("start_positions") for f in features]
        end_positions = [f.pop("end_positions") for f in features]

        # Collate everything else using Hugging Face padding collator
        batch = self.collator(features)

        # Manually add back the start/end positions
        batch["start_positions"] = torch.tensor(start_positions, dtype=torch.long)
        batch["end_positions"] = torch.tensor(end_positions, dtype=torch.long)

        return batch


class DataTrainer:
    def __init__(self, model, dataset_train, dataset_valid, tokenizer, data_collator):
        self.model = model
        self.dataset_train = dataset_train
        self.dataset_valid = dataset_valid
        self.tokenizer = tokenizer
        self.data_collator = data_collator
        self.training_args = TrainingArguments(
            output_dir="results",
            #eval_strategy="epoch",
            eval_strategy="steps",
            eval_steps=1,
            learning_rate=3e-2,
            auto_find_batch_size=True,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=3,
            weight_decay=0.01,
            push_to_hub=False,
            #save_strategy="epoch",
            save_strategy="steps",
            save_steps=1,
            report_to="tensorboard",  # 👈 this disables wandb & enables tensorboard
            logging_dir="logs",
            load_best_model_at_end=True,
        )

        self.trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.dataset_train,
            eval_dataset=self.dataset_valid,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
            compute_metrics=lambda p: compute_metrics(p, self.tokenizer),
        )

    def train(self):
        self.trainer.train()

In [ ]:
collator_distilBert = QADataCollator(distilBerttokenizer)
collator_GPT2 = QADataCollator(GPT2tokenizer)
collator_T5 = QADataCollator(T5Tokenizer)

In [ ]:
trainer_distilBert = DataTrainer(
    model=peft_distilRoberta,
    dataset_train=train_data_distilBert,
    dataset_valid=valid_data_distilBert,
    tokenizer=distilBerttokenizer,
    data_collator=collator_distilBert
)
trainer_distilBert.train()

<ipython-input-34-141bf9caaa6e>:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self.trainer = Trainer(
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


Predictions: (array([[1.5012984, 1.5012985, 1.5012996, ..., 1.5013002, 1.5013002,
        1.5013002],
       [1.4471387, 1.4471387, 1.4471381, ..., 1.4471396, 1.4471396,
        1.4471396],
       [1.4523218, 1.4523267, 1.4523239, ..., 1.4523237, 1.4523237,
        1.4523237],
       ...,
       [1.4576688, 1.457669 , 1.4576696, ..., 1.4576691, 1.4576695,
        1.4576695],
       [1.4392478, 1.439251 , 1.439251 , ..., 1.4392512, 1.4392512,
        1.4392512],
       [1.4436989, 1.4436986, 1.4436991, ..., 1.4436985, 1.4436985,
        1.4436985]], dtype=float32), array([[1.5307624, 1.5307623, 1.5307627, ..., 1.5307628, 1.5307628,
        1.5307628],
       [1.4719981, 1.4719974, 1.4719968, ..., 1.4719979, 1.4719979,
        1.4719979],
       [1.4775965, 1.4775982, 1.4775987, ..., 1.4775981, 1.4775981,
        1.4775981],
       ...,
       [1.4833817, 1.4833821, 1.4833823, ..., 1.4833823, 1.4833826,
        1.4833827],
       [1.4634537, 1.4634575, 1.4634572, ..., 1.4634582, 1.463458

ZeroDivisionError: float division by zero

## Test fine tunned model